In [140]:
#import needed packages 
import os 
import pandas as pd
import numpy as np

In [141]:
#get working directory
os.getcwd()

'c:\\Users\\wilfe\\OneDrive\\Documents\\GitHub\\Ironhack-PRY-BRZ-MGA'

In [142]:
#saving tariff dfs 
py_app_tariffs = pd.read_csv('pry_app_tariffs.csv')
py_bnd_tariffs = pd.read_csv('pry_bnd_tariffs.csv')

#saving ITIC df
py_itic = pd.read_csv('pry_ave_itic.csv')

#saving import/export df
py_imports = pd.read_csv('cleaned PRY imports from BRZ.csv')
br_exports = pd.read_csv('cleaned br exports to py.csv')

In [143]:
#filtering only most recent tariff data for PY 
py_app_tariffs = py_app_tariffs[py_app_tariffs['year']==2021]


In [144]:
#making subsets with only relevant columns for easier merging 
s_py_app_tariffs = py_app_tariffs.loc[:,['hs_code','applied_tariff']]
s_py_bnd_tariffs = py_bnd_tariffs.loc[:,['hs_code','bound_tariff']]
s_py_itic = py_itic.loc[:,['hs_4_code','ave_itic_rate']]

In [145]:
#merging import / export datasets for PY 
mx_compare2 = pd.merge(py_imports,br_exports, on='hs_code', how= 'outer')

In [146]:
mx_compare2.shape

(3484, 20)

In [147]:
#dropping unnecessary or redundant columns 
mx_compare2.drop(['year_y', 'year_x','commodity_y','commodity_x','cif_value'],axis=1,inplace=True)

In [148]:
mx_compare2

,reporter_x,trade_flow_x,partner_x,hs_code,unit_type_x,quantity_x,net_weight_x,fob_value_x,reporter_y,trade_flow_y,partner_y,unit_type_y,quantity_y,net_weight_y,fob_value_y
0,Paraguay,imported_from,Brazil,10121,NaN,0.000,8200.0,75080.65,Brazil,exported_to,Paraguay,u,17.0,8200.0,75081.0
1,Paraguay,imported_from,Brazil,10221,units,79.701,10925.0,163801.19,Brazil,exported_to,Paraguay,u,17.0,10937.0,163801.0
2,Paraguay,imported_from,Brazil,10310,NaN,0.000,55587.0,649040.72,Brazil,exported_to,Paraguay,u,641.0,58244.0,649041.0
3,Paraguay,imported_from,Brazil,10392,units,2690.914,79650.0,579105.90,Brazil,exported_to,Paraguay,u,904.0,79649.0,579106.0
4,Paraguay,imported_from,Brazil,10511,NaN,0.000,667138.8,10969520.83,Brazil,exported_to,Paraguay,u,13417432.0,642645.0,10782248.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3479,NaN,NaN,NaN,910229,NaN,NaN,NaN,NaN,Brazil,exported_to,Paraguay,u,100.0,3.0,60.0
3480,NaN,NaN,NaN,910521,NaN,NaN,NaN,NaN,Brazil,exported_to,Paraguay,u,10.0,4.0,103.0
3481,NaN,NaN,NaN,950450,NaN,NaN,NaN,NaN,Brazil,exported_to,Paraguay,kg,9.0,9.0,957.0
3482,NaN,NaN,NaN,950632,NaN,NaN,NaN,NaN,Brazil,exported_to,Paraguay,u,22.0,14.0,1031.0


In [149]:
#standardizing unit types 
mx_compare2['unit_type_y'] = mx_compare2['unit_type_y'].replace({'u':'units','m²':'square_meters','2u':'pairs_of_units','U (jeu/pack)' : 'unit'})

In [150]:
#renaming columns to better reflect values 
mx_compare2.rename(columns={'quantity_x':'py_m_q','unit_type_x':'py_m_unit','net_weight_x':'py_m_nt_wgt','unit_type_y':'br_x_unit','quantity_y':'br_x_q','net_weight_y':'br_x_nt_wgt','fob_value_x':'py_m_value','fob_value_y':'br_x_value'},inplace=True)

In [151]:
#reordering columns so they make logical sense 
mx_compare2 = mx_compare2[['hs_code', 'br_x_q', 'br_x_unit','py_m_q', 'py_m_unit', 'br_x_nt_wgt','py_m_nt_wgt','br_x_value', 'py_m_value']]

In [152]:
#merging subsets into main dataframe
mx_compare2 = pd.merge(mx_compare2, s_py_app_tariffs, on="hs_code",how="inner")
mx_compare2 = pd.merge(mx_compare2, s_py_bnd_tariffs, on="hs_code",how="inner")

In [153]:
#changing hs_code to string and then adding 0 to the front of the code so all of them are six digits long 
mx_compare2['hs_code'] = mx_compare2['hs_code'].astype(str)
char_to_add = '0'
mx_compare2['hs_code'] = mx_compare2['hs_code'].apply(lambda x : char_to_add + x if len(x) == 5 else x)
mx_compare2

,hs_code,br_x_q,br_x_unit,py_m_q,py_m_unit,br_x_nt_wgt,py_m_nt_wgt,br_x_value,py_m_value,applied_tariff,bound_tariff
0,010310,641.0,units,0.000,NaN,58244.0,55587.0,649041.0,649040.72,0.0,10.0
1,010392,904.0,units,2690.914,units,79649.0,79650.0,579106.0,579105.90,2.0,35.0
2,010511,13417432.0,units,0.000,NaN,642645.0,667138.8,10782248.0,10969520.83,1.0,15.0
3,020130,3428542.0,kg,3828180.000,kg,3428542.0,3828180.0,11463570.0,14030355.31,12.0,30.0
4,020220,1822983.0,kg,1845790.000,kg,1822983.0,1845790.0,6153697.0,6235915.96,10.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...
3056,910211,2.0,units,NaN,NaN,1.0,NaN,580.0,NaN,6.0,25.0
3057,910229,100.0,units,NaN,NaN,3.0,NaN,60.0,NaN,20.0,25.0
3058,910521,10.0,units,NaN,NaN,4.0,NaN,103.0,NaN,6.0,35.0
3059,950632,22.0,units,NaN,NaN,14.0,NaN,1031.0,NaN,6.0,35.0


In [154]:
#changing hs_4_code to string and then adding 0 to the front of the code so all of them are four digits long 
s_py_itic['hs_4_code'] = s_py_itic['hs_4_code'].astype(str)
s_py_itic['hs_4_code'] = s_py_itic['hs_4_code'].apply(lambda x : char_to_add + x if len(x) == 3 else x)

In [155]:
#create a mapping from first four digits and populate with related rate 
mapping = pd.Series(s_py_itic['ave_itic_rate'].values, index=s_py_itic['hs_4_code']).to_dict()

In [156]:
# Define itic_mapping function to populate itic data in main df w/ proper corresponding itic rates.
def itic_mapping(row, mapping):
    first_four = row['hs_code'][:4]
    return mapping.get(first_four, pd.NA)


In [157]:
# Maps itic rates to proper rows based on hs code
mx_compare2['itic_rate'] = mx_compare2.apply(itic_mapping, mapping=mapping, axis=1)

In [158]:
#changing column order 
mx_compare2 = mx_compare2[['hs_code', 'br_x_q', 'br_x_unit','py_m_q', 'py_m_unit', 'br_x_nt_wgt','py_m_nt_wgt','br_x_value', 'itic_rate', 'applied_tariff','bound_tariff','py_m_value']]

In [159]:
#bound tariff column is no longer necessary, so drop
mx_compare2.drop(columns=['bound_tariff'], inplace=True)

C:\Users\wilfe\AppData\Local\Temp\ipykernel_6448\313786313.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mx_compare2.drop(columns=['bound_tariff'], inplace=True)


In [160]:
# Fills null values in most numeric columns with 0
# Fills null values in non-numeric columns with NR, standing for Not Reported
# Fills null vaues in itic_rate column with the average of that column. 
# We chose to fill w/ average b/c if a country's trade patterns tend towards...
# ... more or less expensive shipping, we want that to be reflected in the...
# ... itic values we have to fill. 
mx_compare2['br_x_q'].fillna(0,inplace=True)
mx_compare2['br_x_unit'].fillna('NR',inplace=True)
mx_compare2['py_m_q'].fillna(0,inplace=True)
mx_compare2['py_m_unit'].fillna('NR',inplace=True)
mx_compare2['br_x_nt_wgt'].fillna(0,inplace=True)
mx_compare2['py_m_nt_wgt'].fillna(0,inplace=True)
mx_compare2['br_x_value'].fillna(0, inplace=True)
mx_compare2['itic_rate'].fillna(mx_compare2['itic_rate'].mean(),inplace=True)
mx_compare2['py_m_value'].fillna(0, inplace=True)

C:\Users\wilfe\AppData\Local\Temp\ipykernel_6448\2206341788.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mx_compare2['br_x_q'].fillna(0,inplace=True)
C:\Users\wilfe\AppData\Local\Temp\ipykernel_6448\2206341788.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mx_compare2['br_x_unit'].fillna('NR',inplace=True)
C:\Users\wilfe\AppData\Local\Temp\ipykernel_6448\2206341788.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mx_comp

In [161]:
#made new columns to fill with float values
new_columns = ['adj_x_value','trade_gap','est_tax_loss','wgt_gap','value_ratio','wgt_ratio','density']
for column in new_columns:
    mx_compare2[column]=0.0

In [162]:
#populating new columns with equations for analysis 

# Multiplies reported export value by itic rate to account for shipping cost.
mx_compare2['adj_x_value']=mx_compare2['br_x_value']*mx_compare2['itic_rate']
# Subtracts reported import value from export value to identify value gap. 
mx_compare2['trade_gap']=mx_compare2['adj_x_value']-mx_compare2['py_m_value']

# Multiples trade gap by tariff rate to estimate amount of tariff lost due to gap. 
# This would identify which HS codes need the most attention of gov't action because bigger tariff losses necessitate greater attention.
mx_compare2['est_tax_loss']=mx_compare2['trade_gap']*(mx_compare2['applied_tariff']/100)
# Subtracts reported import weight from import weight to identify weight gap.
mx_compare2['wgt_gap']= mx_compare2['br_x_nt_wgt'] - mx_compare2['py_m_nt_wgt']

# The closer any of the following ratios are to 1, the more consistent reporting was between Brazil and Paraguay. 
#The farther from 1 the bigger a discrepancy there is. 

# Divides value of imports by value of exports to establish a value ratio.
mx_compare2['value_ratio'] = mx_compare2['py_m_value']/mx_compare2['adj_x_value']
# Divides value of imports by value of exports to establish a value ratio.
mx_compare2['wgt_ratio'] = mx_compare2['py_m_nt_wgt']/mx_compare2['br_x_nt_wgt']
# Divides value ratio by weight ratio. 
mx_compare2['density']=mx_compare2['value_ratio']/mx_compare2['wgt_ratio']

In [163]:
#adjusting column order 
mx_compare2 = mx_compare2[['hs_code', 'br_x_q', 'br_x_unit', 'py_m_q', 'py_m_unit', 'br_x_nt_wgt',
       'py_m_nt_wgt', 'br_x_value', 'itic_rate', 'applied_tariff', 'adj_x_value',
       'py_m_value', 'trade_gap', 'est_tax_loss', 'wgt_gap',
       'value_ratio', 'wgt_ratio', 'density']]

In [164]:
#checking for duplicates
mx_compare2.duplicated().sum()

0

In [165]:
#checking for nulls by column 
mx_compare2.isnull().sum()

hs_code             0
br_x_q              0
br_x_unit           0
py_m_q              0
py_m_unit           0
br_x_nt_wgt         0
py_m_nt_wgt         0
br_x_value          0
itic_rate           0
applied_tariff      0
adj_x_value         0
py_m_value          0
trade_gap           0
est_tax_loss        0
wgt_gap             0
value_ratio         0
wgt_ratio           2
density           300
dtype: int64

In [166]:
# Creates and displays null weight column to investigate null value. 
null_wgt = mx_compare2[mx_compare2['wgt_ratio'].isnull() == True]
null_wgt
mx_compare2

,hs_code,br_x_q,br_x_unit,py_m_q,py_m_unit,br_x_nt_wgt,py_m_nt_wgt,br_x_value,itic_rate,applied_tariff,adj_x_value,py_m_value,trade_gap,est_tax_loss,wgt_gap,value_ratio,wgt_ratio,density
0,010310,641.0,units,0.000,NR,58244.0,55587.0,649041.0,1.0836,0.0,7.033008e+05,649040.72,5.426011e+04,0.000000,2657.0,0.922849,0.954382,0.966961
1,010392,904.0,units,2690.914,units,79649.0,79650.0,579106.0,1.0836,2.0,6.275193e+05,579105.90,4.841336e+04,968.267232,-1.0,0.922850,1.000013,0.922838
2,010511,13417432.0,units,0.000,NR,642645.0,667138.8,10782248.0,1.0626,1.0,1.145722e+07,10969520.83,4.876959e+05,4876.958948,-24493.8,0.957433,1.038114,0.922281
3,020130,3428542.0,kg,3828180.000,kg,3428542.0,3828180.0,11463570.0,1.0360,12.0,1.187626e+07,14030355.31,-2.154097e+06,-258491.614800,-399638.0,1.181378,1.116562,1.058050
4,020220,1822983.0,kg,1845790.000,kg,1822983.0,1845790.0,6153697.0,1.0402,10.0,6.401076e+06,6235915.96,1.651597e+05,16515.965940,-22807.0,0.974198,1.012511,0.962161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3056,910211,2.0,units,0.000,NR,1.0,0.0,580.0,1.0280,6.0,5.962400e+02,0.00,5.962400e+02,35.774400,1.0,0.000000,0.000000,NaN
3057,910229,100.0,units,0.000,NR,3.0,0.0,60.0,1.0280,20.0,6.168000e+01,0.00,6.168000e+01,12.336000,3.0,0.000000,0.000000,NaN
3058,910521,10.0,units,0.000,NR,4.0,0.0,103.0,1.0450,6.0,1.076350e+02,0.00,1.076350e+02,6.458100,4.0,0.000000,0.000000,NaN
3059,950632,22.0,units,0.000,NR,14.0,0.0,1031.0,1.0418,6.0,1.074096e+03,0.00,1.074096e+03,64.445748,14.0,0.000000,0.000000,NaN


In [167]:
# Defines columns to round
numeric_columns = ['br_x_q', 'py_m_q',
       'br_x_nt_wgt', 'py_m_nt_wgt', 'br_x_value', 'itic_rate',
       'applied_tariff', 'adj_x_value', 'py_m_value', 'trade_gap',
       'est_tax_loss', 'wgt_gap', 'value_ratio', 'wgt_ratio', 'density']

# Rounds defined columns to improve legibility
mx_compare2[numeric_columns] = mx_compare2[numeric_columns].round(2)

In [168]:
# Replace 0.0s and infs in value_ratio and wgt_ratio columns
replacement_mapping = {0.0: 'NMR'}
mx_compare2 = mx_compare2.replace({'value_ratio': replacement_mapping, 'wgt_ratio': replacement_mapping})

inf_columns = ['value_ratio','wgt_ratio']
mx_compare2[inf_columns] = mx_compare2[inf_columns].replace([np.inf,-np.inf],'NXR')


In [169]:
#displays final, recovered cleaned mx_compare2 dataset
mx_compare2

,hs_code,br_x_q,br_x_unit,py_m_q,py_m_unit,br_x_nt_wgt,py_m_nt_wgt,br_x_value,itic_rate,applied_tariff,adj_x_value,py_m_value,trade_gap,est_tax_loss,wgt_gap,value_ratio,wgt_ratio,density
0,010310,641.0,units,0.00,NR,58244.0,55587.0,649041.0,1.08,0.0,703300.83,649040.72,54260.11,0.00,2657.0,0.92,0.95,0.97
1,010392,904.0,units,2690.91,units,79649.0,79650.0,579106.0,1.08,2.0,627519.26,579105.90,48413.36,968.27,-1.0,0.92,1.0,0.92
2,010511,13417432.0,units,0.00,NR,642645.0,667138.8,10782248.0,1.06,1.0,11457216.72,10969520.83,487695.89,4876.96,-24493.8,0.96,1.04,0.92
3,020130,3428542.0,kg,3828180.00,kg,3428542.0,3828180.0,11463570.0,1.04,12.0,11876258.52,14030355.31,-2154096.79,-258491.61,-399638.0,1.18,1.12,1.06
4,020220,1822983.0,kg,1845790.00,kg,1822983.0,1845790.0,6153697.0,1.04,10.0,6401075.62,6235915.96,165159.66,16515.97,-22807.0,0.97,1.01,0.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3056,910211,2.0,units,0.00,NR,1.0,0.0,580.0,1.03,6.0,596.24,0.00,596.24,35.77,1.0,NMR,NMR,NaN
3057,910229,100.0,units,0.00,NR,3.0,0.0,60.0,1.03,20.0,61.68,0.00,61.68,12.34,3.0,NMR,NMR,NaN
3058,910521,10.0,units,0.00,NR,4.0,0.0,103.0,1.04,6.0,107.64,0.00,107.64,6.46,4.0,NMR,NMR,NaN
3059,950632,22.0,units,0.00,NR,14.0,0.0,1031.0,1.04,6.0,1074.10,0.00,1074.10,64.45,14.0,NMR,NMR,NaN


In [ ]:
# Saves cleaned dataframe as a new .csv file for future use. 
# mx_compare3.to_csv('mx_compare2.csv',index=False)

#not saving because we are using original csv, this is the recovered 
#copy of Maya's notebook that demonstrates working code 